<a href="https://colab.research.google.com/github/ronbalanay/MAT-422/blob/main/MAT422_HW3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.3.1. Necessary and sufficient conditions of local minimizers

We check whether a point is a global and local minimizer for the function f(x)=∑x_i^2 using the gradient and Hessian. We verifies the first-order necessary condition (gradient is zero) and the second-order sufficient condition (Hessian is positive definite). The function is printed, and tests are run to identify if the point is a minimizer.


In [1]:
import numpy as np
from scipy.optimize import minimize
from numpy.linalg import eigvals

# define the function f(x)
def f(x):
    return np.sum(x**2)  # example function: f(x) = sum(x_i^2), which is convex

# gradient of the function f(x)
def grad_f(x):
    return 2 * x  # gradient: ∇f(x) = 2*x

# hessian of the function f(x)
def hessian_f(x):
    return 2 * np.eye(len(x))  # hessian: 2*identity matrix (positive definite)

# check for global minimizer
def is_global_minimizer(x_star):
    # a convex function like sum(x_i^2) has a global minimum at x = 0
    f_x_star = f(x_star)
    for _ in range(1000):
        x_rand = np.random.uniform(-10, 10, len(x_star))
        if f(x_rand) < f_x_star:
            return False
    return True

# check for local minimizer using first-order necessary condition
def is_local_minimizer(x_star):
    grad = grad_f(x_star)
    if np.allclose(grad, np.zeros_like(grad), atol=1e-5):
        return True
    return False

# second-order sufficient condition (check positive definiteness of hessian)
def second_order_sufficient_condition(x_star):
    hessian = hessian_f(x_star)
    eigenvalues = eigvals(hessian)
    if np.all(eigenvalues > 0):  # positive definite hessian
        return True
    return False

# print the function being referenced
def print_function():
    print("Function f(x): sum(x_i^2)")

# test point for minimization
x_star = np.array([0.0, 0.0])

# print the function we are referencing
print_function()

# global minimizer check
if is_global_minimizer(x_star):
    print(f"{x_star} is a global minimizer.")
else:
    print(f"{x_star} is NOT a global minimizer.")

# local minimizer check
if is_local_minimizer(x_star):
    print(f"{x_star} satisfies the first-order necessary condition for a local minimizer.")
else:
    print(f"{x_star} does NOT satisfy the first-order necessary condition for a local minimizer.")

# second-order sufficient condition check
if second_order_sufficient_condition(x_star):
    print(f"{x_star} satisfies the second-order sufficient condition and is a strict local minimizer.")
else:
    print(f"{x_star} does NOT satisfy the second-order sufficient condition.")


Function f(x): sum(x_i^2)
[0. 0.] is a global minimizer.
[0. 0.] satisfies the first-order necessary condition for a local minimizer.
[0. 0.] satisfies the second-order sufficient condition and is a strict local minimizer.


# 3.3.2. Convexity and global minimizers
We check whether a set of points forms a convex set and whether an open ball in R^2 is convex. We verify the convexity by checking if any convex combination of two points in the set or ball still lies within the set or ball. The set of points is printed, and tests are run to determine if the set and the open ball are convex.

In [2]:
import numpy as np

# function to check if a set D is convex
def is_convex_set(D):
    """
    Check if the set D ⊆ R^d is convex.

    Arguments:
    D : list of numpy arrays - Points that belong to set D.

    Returns:
    bool : True if D is convex, False otherwise.
    """
    for i, x in enumerate(D):
        for j, y in enumerate(D):
            if i != j:
                # check for all alpha in [0, 1]
                alpha_values = np.linspace(0, 1, 100)
                for alpha in alpha_values:
                    z = (1 - alpha) * x + alpha * y
                    if not any(np.allclose(z, point) for point in D):
                        return False
    return True

# example: Open ball in R^d is convex
def open_ball_is_convex(x0, delta, points):
    """
    Check if an open ball B_delta(x0) is convex.

    Arguments:
    x0 : numpy array - Center of the open ball.
    delta : float - Radius of the open ball.
    points : list of numpy arrays - Points that belong to the open ball.

    Returns:
    bool : True if B_delta(x0) is convex, False otherwise.
    """
    for x in points:
        for y in points:
            for alpha in np.linspace(0, 1, 100):
                w = (1 - alpha) * x + alpha * y
                if np.linalg.norm(w - x0) > delta:
                    return False
    return True

# test case: Points in a convex set (e.g., unit ball in R^2)
x0 = np.array([0, 0])
delta = 1
points_in_ball = [np.array([0.5, 0.5]), np.array([0.3, -0.7]), np.array([-0.6, 0.6])]

# print the set we are working with
print("Points in the set we are working with:", points_in_ball)

# check if the points form a convex set and if the open ball is convex
is_convex = is_convex_set(points_in_ball)
is_ball_convex = open_ball_is_convex(x0, delta, points_in_ball)

print("Is the set convex?", is_convex)
print("Is the open ball convex?", is_ball_convex)


Points in the set we are working with: [array([0.5, 0.5]), array([ 0.3, -0.7]), array([-0.6,  0.6])]
Is the set convex? False
Is the open ball convex? True


# 3.3.3. Gradient descent

We use the gradient descent algorithm to find the local minimum of the function f(x)=x^TAx−b^Tx, where A is a matrix and b is a vector. We start at an initial point and iteratively update the point in the direction of the negative gradient. The gradient is computed at each iteration using the gradient_f function, and the point is updated based on the learning rate. The process continues until the gradient is sufficiently small or the maximum number of iterations is reached. The final output provides the coordinates of the global minimizer and the function's value at that point.

In [3]:
import numpy as np

# define the function f(x) for minimization
def f(x):
    A = np.array([[3, 2], [2, 6]])  # example matrix A
    b = np.array([2, -8])            # example vector b
    return np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)

# compute the gradient of f(x)
def gradient_f(x):
    A = np.array([[3, 2], [2, 6]])  # example matrix A
    b = np.array([2, -8])            # example vector b
    return np.dot(A, x) - b

# gradient descent algorithm
def gradient_descent(starting_point, learning_rate, tolerance, max_iterations):
    x_k = starting_point
    for k in range(max_iterations):
        grad = gradient_f(x_k)  # calculate the gradient at the current point
        if np.linalg.norm(grad) < tolerance:  # check if the gradient is close to zero
            break
        x_k = x_k - learning_rate * grad  # update the point
    return x_k

# parameters for gradient descent
starting_point = np.array([0.0, 0.0])  # initial guess
learning_rate = 0.1                    # step size
tolerance = 1e-6                        # convergence tolerance
max_iterations = 100                    # maximum number of iterations

# execute gradient descent
minimizer = gradient_descent(starting_point, learning_rate, tolerance, max_iterations)

# output the results
print("Function value at minimizer:", f(minimizer))
print("Global minimizer:", minimizer)
print("Matrix A:\n", np.array([[3, 2], [2, 6]]))
print("Vector b:", np.array([2, -8]))


Function value at minimizer: -2.3695261717193716e-06
Global minimizer: [ 1.99999961 -1.9999998 ]
Matrix A:
 [[3 2]
 [2 6]]
Vector b: [ 2 -8]
